<a href="https://colab.research.google.com/github/mclellanw00/module2_intro_to_pandas/blob/main/2_2_dataframe_verbs_select_filter_mutate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Be sure you installed dfply
!pip install dfply

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 612 kB 28.9 MB/s 


## <font color="red"> With the currect version of anaconda3 [2022.05], this won't work on newer M1 Mac's.  If this describes you, please skip over all R code/installations.</font>

In [2]:
#First, make sure you have R installed ... this could take a while ;P
!brew install R

/bin/bash: brew: command not found


In [3]:
# Next, we install rpy2 to allow running R code in a notebook
!pip install rpy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
#Load rpy2 and R magic commands
import rpy2
%load_ext rpy2.ipython

# Select, Filter, and Mutate

In this lecture, we will look at three important actions used to process data frames.  While each framework uses different names for these functions, we will use the names from the `R` library `dplyr`, namely `select`, `mutate`, and `filter`.  The most important takeaway will be that, regardless of framework or scale, we can process data frames in the same way by applying the same sequence of data verbs.

## R and Python can interact!

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
%%R
rnorm(5, 2, 3)

[1]  5.134977  2.441341  1.615844  1.335282 -2.367666


## We love dplyr!

In [9]:
%%R 
library(dplyr)
artists <- read.csv('./data/Artists.csv')

(artists %>%
  select(BeginDate, 
         DisplayName, 
         Nationality) %>%
  filter(BeginDate > 0) %>%
  head) -> output
output


Error in file(file, "rt") : cannot open the connection


RInterpreterError: ignored

## What makes `dplyr` so great?

* Focus on data verbs
* Pipes lead to code that is
    * More readable
    * Easy to compose and debug

## Set up

Let's read in a data set in each of the three frameworks

In [12]:
import pandas as pd
from dfply import *
heroes = pd.read_csv('https://raw.githubusercontent.com/mclellanw00/module2_intro_to_pandas/main/data/heroes_information.csv')
heroes.head()

,Unnamed: 0,name,Gender,Eye color,Race,Hair color,Height,Publisher,Skin color,Alignment,Weight
0,0,A-Bomb,Male,yellow,Human,No Hair,203.0,Marvel Comics,-,good,441.0
1,1,Abe Sapien,Male,blue,Icthyo Sapien,No Hair,191.0,Dark Horse Comics,blue,good,65.0
2,2,Abin Sur,Male,blue,Ungaran,No Hair,185.0,DC Comics,red,good,90.0
3,3,Abomination,Male,green,Human / Radiation,No Hair,203.0,Marvel Comics,-,bad,441.0
4,4,Abraxas,Male,blue,Cosmic Entity,Black,-99.0,Marvel Comics,-,bad,-99.0


In [28]:
heroes.dtypes

Unnamed: 0      int64
name           object
Gender         object
Eye color      object
Race           object
Hair color     object
Height        float64
Publisher      object
Skin color     object
Alignment      object
Weight        float64
dtype: object

## Selecting Columns

<img src="https://github.com/mclellanw00/module2_intro_to_pandas/blob/main/img/select.png?raw=1">

The first verb, `select` 

* filters the *columns*
* At the core of `SQL` statements

## How to select
 pipe (`>>`) into `select` and use `X.column_name` or `X['column name']`

In [13]:
(heroes
 >> select(X.name, 
           X['Gender'],
           'Eye color'
          )
 >> head
)

,name,Gender,Eye color
0,A-Bomb,Male,yellow
1,Abe Sapien,Male,blue
2,Abin Sur,Male,blue
3,Abomination,Male,green
4,Abraxas,Male,blue


## Filtering Rows

<img src="https://github.com/mclellanw00/module2_intro_to_pandas/blob/main/img/filter.png?raw=1">

The next verb, `filter` 

* filters the *rows*
* is related to the `SQL` `WHERE` clause

## How to filter

* pipe (`>>`) into `filter_by` 
* First argument is a boolean expression
* Reference columns with `X.column_name` or `X['column name']`

In [14]:
(heroes 
 >> filter_by(X.Gender == 'Male') 
 >> head
)

,Unnamed: 0,name,Gender,Eye color,Race,Hair color,Height,Publisher,Skin color,Alignment,Weight
0,0,A-Bomb,Male,yellow,Human,No Hair,203.0,Marvel Comics,-,good,441.0
1,1,Abe Sapien,Male,blue,Icthyo Sapien,No Hair,191.0,Dark Horse Comics,blue,good,65.0
2,2,Abin Sur,Male,blue,Ungaran,No Hair,185.0,DC Comics,red,good,90.0
3,3,Abomination,Male,green,Human / Radiation,No Hair,203.0,Marvel Comics,-,bad,441.0
4,4,Abraxas,Male,blue,Cosmic Entity,Black,-99.0,Marvel Comics,-,bad,-99.0


## Chaining Data Verbs

* Processing df $\rightarrow$ chaining data verbs
* Accomplished through pipes/dot-chains

## Example 1 - `select` + `filter`

In [15]:
(heroes 
 >> filter_by(X.Gender == 'Male') 
 >> select(X.name, X.Gender, X.Weight) 
 >> head
)

,name,Gender,Weight
0,A-Bomb,Male,441.0
1,Abe Sapien,Male,65.0
2,Abin Sur,Male,90.0
3,Abomination,Male,441.0
4,Abraxas,Male,-99.0


## Example 2 - `filter` + `filter`

Note that chaining `filter`s is an `and` operation.

####  `pandas` + `dfply`

In [16]:
(heroes >>
   select(X.name, X.Gender, X.Weight) >>
   filter_by(X.Gender == 'Male') >>
   filter_by(X.Weight > 0) >>
   head)

,name,Gender,Weight
0,A-Bomb,Male,441.0
1,Abe Sapien,Male,65.0
2,Abin Sur,Male,90.0
3,Abomination,Male,441.0
5,Absorbing Man,Male,122.0


## <font color="red"> Exercise 2.2.1: Blue-eyed Heroes </font>

Create a query that

1. Selects the name, Gender, and Eye Color columns
2. Filters on eye_color == 'blue'

In [37]:
(heroes >>
   select(X.name, X.Gender, X['Eye color']) >>
   filter_by(X['Eye color'] == 'blue') >>
   head)


,name,Gender,Eye color
1,Abe Sapien,Male,blue
2,Abin Sur,Male,blue
4,Abraxas,Male,blue
5,Absorbing Man,Male,blue
6,Adam Monroe,Male,blue


## Constructing New Columns

The third verb, `mutate` 

* Creates new columns
* Changes existing columns

## How to mutate

*  pipe (`>>`) into `mutate`
* First argument is a transformational expression
* Reference columns with `X.column_name` or `X['column name']`

## Example 3 - Converting Weight to kilograms

Currently, the weight column is in pounds.  Let's convert to kilograms.

In [38]:
(heroes 
 >> select(X.name, 
           X.Gender, 
           X.Weight) 
 >> mutate(Weight_kg = X.Weight/2.2046) 
 >> head
)

,name,Gender,Weight,Weight_kg
0,A-Bomb,Male,441.0,200.036288
1,Abe Sapien,Male,65.0,29.483807
2,Abin Sur,Male,90.0,40.823732
3,Abomination,Male,441.0,200.036288
4,Abraxas,Male,-99.0,-44.906105


## Referencing a new column

Each framework provides a way to reference a new column.

* **Create:** Use `mutate(new_col = ...)`
* **Later reference:** Use `X.new_col` or `X['new_col']`

## Example 4 - Converting Weight to kilograms and filter

Let's find all heroes with a weight under 100kg.

In [39]:
(heroes 
 >> select(X.name, X.Gender, X.Weight) 
 >> mutate(Weight_kg = X.Weight/2.2046) 
 >> filter_by(X.Weight_kg < 100) 
 >> head
)

,name,Gender,Weight,Weight_kg
1,Abe Sapien,Male,65.0,29.483807
2,Abin Sur,Male,90.0,40.823732
4,Abraxas,Male,-99.0,-44.906105
5,Absorbing Man,Male,122.0,55.338837
6,Adam Monroe,Male,-99.0,-44.906105


## <font color="red"> Exercise 2.2.2: Tall Heroes </font>

Create a query that

1. Selects the name, Gender, and Height columns
2. Compute the height in inches.
    * Check [here](https://www.kaggle.com/claudiodavi/superhero-set) to determine the current units.
3. Filters on height_in > 72

In [45]:
# Your code here

(heroes >>
   select(X['name'],
            'Gender',
            'Height')>>
   mutate(height_in = X['Height']/2.54)>>
   filter_by(X['height_in'] > 72))

,name,Gender,Height,height_in
0,A-Bomb,Male,203.0,79.921260
1,Abe Sapien,Male,191.0,75.196850
2,Abin Sur,Male,185.0,72.834646
3,Abomination,Male,203.0,79.921260
5,Absorbing Man,Male,193.0,75.984252
...,...,...,...,...
724,Wyatt Wingfoot,Male,196.0,77.165354
727,Yellow Claw,Male,188.0,74.015748
728,Yellowjacket,Male,183.0,72.047244
730,Ymir,Male,304.8,120.000000
